In [4]:
import pathlib

import numpy as np
import pandas as pd
import tqdm

In [5]:
CHUNKSIZE = 1e5

In [7]:
base_df = pd.read_csv(
    "../data/База по всем организациям и ИП.zip",
    sep=";", 
    chunksize=CHUNKSIZE, 
    dtype=str
)

In [ ]:
out_file = pathlib.Path("/home/pavel/search_base.csv")

for chunk in tqdm.tqdm(base_df):
    try:
        chunk["name"] = chunk["Наименование полное"]
        chunk["tax_number"] = chunk["ИНН"]
        chunk["individual"] = chunk["ОКОПФ (расшифровка)"] == "Индивидуальный предприниматель"
        chunk["creation_date"] = pd.to_datetime(chunk["Дата создания"], errors="coerce").dt.date
        chunk["active"] = chunk["Компания действующая (1) или нет (0)"].fillna(0).astype(bool)
        chunk["activity_code"] = chunk["ОКВЭД2"]
        chunk["address"] = chunk["Юр адрес"]
        
        chunk.dropna(subset="tax_number", inplace=True)
        chunk.dropna(subset="tax_number", inplace=True)
        
        chunk = chunk[[
            "name", "tax_number", "individual", "creation_date",
            "active", "activity_code", "address"
        ]]

        if out_file.exists():
            chunk.to_csv(out_file, header=False, index=False, mode="a")
        else:
            chunk.to_csv(out_file, header=True, index=False, mode="w")
    except Exception as e:
        pass

73it [04:31,  2.98s/it]